In [2]:
!pip install xgboost scikit-learn joblib pandas numpy fastapi uvicorn pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
file_path='/content/drive/MyDrive/Laptop_price.csv'
df = pd.read_csv(file_path)
X = df.drop(columns=['Price'])
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
import joblib

In [11]:
num_features = X.select_dtypes (include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()
num_transformer = Pipeline ([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline ([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5))
])
pipeline.fit(X_train, y_train)
joblib.dump(pipeline, '/content/drive/MyDrive/laptop_price_model.pkl')

['/content/drive/MyDrive/laptop_price_model.pkl']

In [13]:
!git init


Reinitialized existing Git repository in /content/.git/


In [19]:
!cp /content/drive/MyDrive/Laptop_price.csv /content/MTUCI


In [20]:
%cd /content/MTUCI

/content/MTUCI


In [22]:
!git config --global user.name m4yth11
!git config --global user.email cleindass@gmail.com

In [24]:
!git add Laptop_price.csv
!git commit -m "Добавлен файл"

[main (root-commit) 6573048] Добавлен файл
 1 file changed, 1001 insertions(+)
 create mode 100644 Laptop_price.csv


In [33]:
!git push -u origin main

fatal: could not read Username for 'https://github.com': No such device or address


In [39]:
%%writefile app.pipeline
from fastapi import FastAPI, File, UploadFile
import pandas as pd
import joblib
from io import BytesIO
app = FastAPI()
model_path="/content/drive/MyDrive/laptop_price_model.pkl"
model=joblib.load(model_path)
@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
 content = await file.read()
 df = pd.read_csv(BytesIO(content))
 predictions = model.predict(df)
 return{"predictions": predictions.tolist()}

Overwriting app.pipeline


In [40]:
!pip install python-multipart

In [41]:
!ngrok config add-authtoken 2x5KGgOpZXzeisMNFkkF2QbGYKm_6FES73o7Kek5rArqrMME8

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [42]:
!nohup uvicorn app:app host 0.0.0.0 --port 8000 --reload > fastapi.log 2>&1&

In [43]:
from pyngrok import ngrok

In [44]:
public_url = ngrok.connect(8000)
print("API доступно по адресу:", public_url)

API доступно по адресу: NgrokTunnel: "https://9108-34-44-193-108.ngrok-free.app" -> "http://localhost:8000"
